In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
base_url = "https://www.propertypro.ng/property-for-short-let"
headers = {'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Mobile Safari/537.36'}

In [6]:
response = requests.get(base_url, headers=headers)
response.status_code
# response.text

200

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.title.get_text())

Short let property  (1,774 listings) | PropertyPro.ng


In [9]:
titles = soup.select('.pro-main-cont > a > h2')
len(titles)

20

In [11]:
titles_list = []
for title in titles:
    titles_list.append(title.get_text())
titles_list[1]

'3 BEDROOM FLAT / APARTMENT FOR SHORTLET - OLD IKOYI IKOYI LAGOS'

In [12]:
prices = soup.select('.pro-main-cont > p > span')
len(prices)

40

In [13]:
prices_list = []
for i, price in enumerate(prices):
    if i % 2 == 1:
        prices_list.append(price.get("content"))
prices_list[0]        

'45000.0'

In [14]:
locations = soup.select('.pro-location')

locations_list = []
for location in locations:
    locations_list.append(location.get_text())

In [15]:
property = pd.DataFrame({"title": titles_list, 
                         "price": prices_list,
                         "location": locations_list})

In [16]:
property.to_csv("property.csv")

In [47]:
def scrape_page(url):
    """
        Argument: url -> of the website we want to scrape
        Output: (titles_list, prices_list, locations_list)
    """
    r = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')

    titles = soup.select('.pro-main-cont > a > h2')
    titles_list = []
    for title in titles:
        titles_list.append(title.get_text())

    prices = soup.select('.pro-main-cont > p > span')
    prices_list = []
    for i, price in enumerate(prices):
        if i % 2 != 0:
            prices_list.append(price.get("content"))
    
    locations = soup.select('.pro-location')
    locations_list = []
    for location in locations:
        locations_list.append(location.get_text())
    
    return titles_list, prices_list, locations_list

In [43]:
# urls = ["https://www.propertypro.ng/property-for-short-let", "https://www.propertypro.ng/property-for-short-let?page=1",
# "https://www.propertypro.ng/property-for-short-let?page=2",
# ........, "https://www.propertypro.ng/property-for-short-let?page=44"]

base_url = "https://www.propertypro.ng/property-for-short-let"
urls = [base_url]
for i in range(1,45):
    url = base_url + "?page=" + str(i)
    urls.append(url)

In [46]:
urls[44]

'https://www.propertypro.ng/property-for-short-let?page=44'

In [51]:
master_titles, master_prices, master_locations = [], [], []
for url in urls:
    titles_list, prices_list, locations_list = scrape_page(url)
    master_titles += titles_list
    master_prices += prices_list
    master_locations += locations_list
    """
Prevent DOS attack by resting a bit
Advance usage : Also rotate proxies with this tutorial 
https://www.scrapehero.comhow-to-rotate-proxies-and-ip-addresses-using-python-3/
    """
    time.sleep(30)


In [54]:
len(master_locations)

900

In [55]:
property = pd.DataFrame({"title": master_titles, 
                         "price": master_prices,
                         "location": master_locations})

In [56]:
property.to_csv("property.csv")